### Problem Statement:
- **To analyze customer data from an online retail plateform to segment customers based on their purchasing behaviour, preferences and demographics. The goal is to gain insight into distint customer segments and tailor marketing stratiegies to target each segment effectivelly.**

In [1]:
## Importing all the essential libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

**Loading the dataset**

In [2]:
df = pd.read_excel('Online Retail.xlsx')

In [3]:
### top 5 records
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


**Attribute Information:**

- **InvoiceNo:** Invoice number A 6-digit integral number uniquely assigned to each customers,If this code starts with letter 'c', it indicates a cancellation.
- **StockCode:** Product (item) code, a 5-digit integral number uniquely assigned to each  distinct product.
- **Description:** product item descriptions.
- **Quantity:** The quantities of each product (item) per transaction.
- **InvoiceDate:** Invice date and time. The day and time when a transaction was generated.
- **UnitPrice:** Unit price. Product price per unit.
- **CustomerID:** Customer number. A 5-digit integral number uniquely assigned to each customer.
- **Country:** Country name. The name of the country where a customer resides.

**Exploring the dataset**

In [4]:
df.shape ## dataset rows and columns

(541909, 8)

In [5]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [6]:
df.dtypes ## datatypes of all columns

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [7]:
df.info() #gives all info about the dataset.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [8]:
## Checking missing values
df.isnull().sum().sort_values(ascending=False)

CustomerID     135080
Description      1454
InvoiceNo           0
StockCode           0
Quantity            0
InvoiceDate         0
UnitPrice           0
Country             0
dtype: int64

In [9]:
(df.isnull().sum()/len(df)*100).sort_values(ascending=False)  ## % of missing values

CustomerID     24.926694
Description     0.268311
InvoiceNo       0.000000
StockCode       0.000000
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
Country         0.000000
dtype: float64

- For purposes of customer segmentation missing ids, we  will dropped missing values in customers id.

In [10]:
print("Number of unique customers IDs:", len(df['CustomerID'].unique()))

Number of unique customers IDs: 4373


In [11]:
df = df.dropna()

In [12]:
df.shape

(406829, 8)

- Now we have 406829 rows and 8 columns in our dataset.

In [13]:
df['Description'] = df['Description'].str.lower()

In [14]:
df.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,white metal lantern,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [15]:
## Converting customer_id(float type) to integer type
df['CustomerID'] = df['CustomerID'].astype('int64') 

In [16]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


- As we can see,some quantity is negative, but quantity values should ideally be positive in nature.

- Let's have a closer look on the reason of negative values in these variables. In the description of dataset we can see  that sme trancsaction are started with letter 'C',which means Cancled transactions. Probably transactions with negative values of quantity are canceled.

In [17]:
## Filtering transaction with invoice number started with letter 'C'
cancelled_trans = df[df['InvoiceNo'].astype(str).str.contains('C')]
cancelled_trans.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
154,C536383,35004C,set of 3 coloured flying ducks,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,plasters in tin circus parade,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,pack of 12 pink paisley tissues,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,pack of 12 blue paisley tissues,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom


In [18]:
cancelled_trans['InvoiceNo'].count()

8905

In [19]:
cancelled_trans[cancelled_trans['Quantity']>0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


- We can see there are 8905 InvoiceNo with the Cancelled transaction.
- Among cancelled transactions ther are no transaction with positive quantity

In [20]:
print("Number of cancelled transactions:", len(cancelled_trans))
print("Total number of transaction with negative quantity:", df[df['Quantity'] < 0 ]['Quantity'].count())

Number of cancelled transactions: 8905
Total number of transaction with negative quantity: 8905


- No negative quantity in dataset.
- Assuming that quantity should not be negative we will just drop the InvoiceID with cancelled transaction.

In [21]:
pos_quan = df['Quantity']>0

In [22]:
df = df[pos_quan]

In [23]:
## dataset shape after removing negative quantities

df.shape

(397924, 8)

In [24]:
df.describe().round(2)

,Quantity,UnitPrice,CustomerID
count,397924.00,397924.00,397924.00
mean,13.02,3.12,15294.32
std,180.42,22.10,1713.17
min,1.00,0.00,12346.00
25%,2.00,1.25,13969.00
50%,6.00,1.95,15159.00
75%,12.00,3.75,16795.00
max,80995.00,8142.75,18287.00


In [25]:
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom


In [26]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID              int64
Country                object
dtype: object

- In Pandas, the day format starts from 0 to 6, i.e. Monday=0, Tuesday=1,.....Sunday=6, so apply +1 to make Monday=1...untill Sunday=7

In [ ]:
df["Year"] = (df["InvoiceDate"]).dt.year
df["Month"] = (df["InvoiceDate"]).dt.month
df["Day"] = ((df["InvoiceDate"]).dt.dayofweek) + 1
df["Hour"] = df["InvoiceDate"].dt.strftime('%H')
df.head(3)

In [ ]:
df['Day_Name'] = df['InvoiceDate'].dt.day_name()
df.head(3)

In [ ]:
df["IsWeekend"] = (df["Day"] >= 6).astype(int)
df.head(3)

In [ ]:
df['Year'].unique()

In [ ]:
df['Day'].unique()

In [ ]:
df['Day_Name'].unique()

**creating a new feature, Total_Amount which the  Customer spend on Purchase**

In [ ]:
df['Amount_Spent'] = df['Quantity']* df['UnitPrice']
df.head(3)

### Data Visulaization(EDA)

**Top 10 cutomers with Country and Invoice Number**

In [ ]:
cust_invoice_country = df.groupby(['CustomerID','Country'])['InvoiceNo'].count().sort_values(ascending=False)[:10].reset_index()
cust_invoice_country

In [ ]:
customer_invoice = df.groupby(['CustomerID'])['InvoiceNo'].count().sort_values(ascending=False)[:10]

In [ ]:
plt.figure(figsize=(6,5))
ax = sns.barplot(x=customer_invoice.index, y=customer_invoice.values)
plt.xlabel('Customer ID')
plt.ylabel('Number of Orders')
plt.title('Number of Orders for different Customers')
plt.xticks(rotation= 45)
for label in ax.containers:
    ax.bar_label(label);

In [ ]:
country_invoice = df.groupby(['Country'])['InvoiceNo'].count().sort_values(ascending=False)[:10]
country_invoice

In [ ]:
plt.figure(figsize=(6,5))
ax = sns.barplot(x=country_invoice.index, y=country_invoice.values)
plt.xlabel('Country')
plt.ylabel('Number of Orders')
plt.title('Customers country Vs InvoiceNo')
plt.xticks(rotation=45)
for label in ax.containers:
    ax.bar_label(label)

plt.show() 

- We can see that most of the InvoiceNo generated for United Kingdom which means most of the customers are from United Kingdom.

* **Total_Amount spent by top 10 Customers based on invoiceNo and Country.**

In [ ]:
money_spent_10 = df.groupby(by=['CustomerID','Country','InvoiceNo'])['Amount_Spent'].sum().sort_values(ascending=False).reset_index()[:10]
money_spent_10

In [ ]:
money_spent = df.groupby(by=['CustomerID'])['Amount_Spent'].sum().sort_values(ascending=False).reset_index()[:20]
money_spent

In [ ]:
plt.figure(figsize = (12,5))
ax = sns.barplot(x = 'CustomerID',  y = 'Amount_Spent', data= money_spent)
plt.xlabel('Customers ID')
plt.ylabel('Amount Spent')
plt.title('Top 20 CustomersID Vs Amount Spent')
plt.xticks(rotation=45)
for label in ax.containers:
    ax.bar_label(label)

plt.show() 

**How many orders per month?**

In [ ]:
monthly_sale = df.groupby(['Month'])['InvoiceNo'].count().reindex([1,2,3,4,5,6,7,8,9,10,11,12]).reset_index()
monthly_sale

In [ ]:
df['Month'].unique()

In [ ]:
plt.figure(figsize=(4,3))
ax = sns.lineplot(x="Month", y="InvoiceNo", data = monthly_sale )
plt.xlabel("Month")
plt.ylabel("Invoice generated")
plt.title("Month wise Orders")
plt.show()

- There are more purchasing made at the end of the year. The highest number of transactions were made during November. Probably reason could be the fact, that people tend to prepare for Christmas during November. Other than the month of November, number of purchases during the year is quite similar.

**How many orders per Day?**

In [ ]:
daily_sale = df.groupby(['Day_Name'])['InvoiceNo'].count().reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Sunday']).reset_index()
daily_sale

In [ ]:
plt.figure(figsize=(5,4))
ax = sns.barplot(x="Day_Name", y="InvoiceNo", data =daily_sale )
plt.xticks(rotation=45)
for label in ax.containers:
    ax.bar_label(label)
plt.show()

- We can see that, on ThrusDay there is somewhat more sale as compare to other Days
- Interesting fact is that there were not any transaction on Saturday during the whole period. One of the explanations colud be some filter during the data collection process. Number of buyings are not significantly differ according to the day of the week.

**Orders per hour**

In [ ]:
hourly_sale = df.groupby(['Hour'])['InvoiceNo'].count().reset_index()
hourly_sale

In [ ]:
ax = sns.barplot(x="Hour", y= 'InvoiceNo', data =hourly_sale )
plt.title('Hourly Sale')
for label in ax.containers:
    ax.bar_label(label)
plt.show()

- The most number of transactions is done between 12 a.m. and 2 p.m., There aren't any transactions after 8 p.m. till 6 a.m.

In [ ]:
### Statisticals Summary of UnitPrice
df['UnitPrice'].describe().round(2)

- From the descriptive statistics for UnitPrice variable, we could see that most ot the sold products are price are quite cheap.
- Now lets see the UnitPrice of products with Description, which unit price is more than 50.

In [ ]:
df[df['UnitPrice']>100]['Description'].unique()

In [ ]:
df[df['UnitPrice']>1000]['Description'].unique()

- 3 products which Unitprice is more than 1000.

In [ ]:
df[df['UnitPrice']>5000]['Description'].unique()

- There are only one product named 'postage' which is more costly.

In [ ]:
plt.figure(figsize=(4,3))
sns.distplot(df[df['UnitPrice']>50]['UnitPrice'], kde=True)
plt.show()

In [ ]:
plt.figure(figsize=(4,3))
sns.distplot(df[df['UnitPrice']>500]['UnitPrice'], kde=True)
plt.show()

In [ ]:
df['Country'].unique()

In [ ]:
print("Transactions were generated in", len(df['Country'].unique()), "different countries")

In [ ]:
df[df['Country']=='Unspecified'].count()[6]

In [ ]:
print("Number of transactions where country is unspecified:", len(df[df['Country']=='Unspecified']))

###  Adding new attributes.
- RFM (Recency, Frequency, Monetary) analysis is a customer segmentation technique that uses past purchase behavior to divide customers into groups. RFM helps divide customers into various categories or clusters to identify customers who are more likely to respond to promotions and also for future personalization services.

- **Recency(R)**: Days since last purchase
- **Frequency(F)**: Total number of purchases
- **Monetory value(M)**: Total money, customer spent.

In [ ]:
## Calculating Monetary(Amount_Spent)
rfm_m = df.groupby('CustomerID')['Amount_Spent'].sum().reset_index().rename(columns={'Amount_Spent':'Amount'})
rfm_m.head()

In [ ]:
## Calculating Frequency.
rfm_f = df.groupby('CustomerID')['InvoiceNo'].count().reset_index().rename(columns= {'InvoiceNo':'Frequency'})
rfm_f.head()

In [ ]:
rfm = pd.merge(rfm_m, rfm_f, on='CustomerID', how='inner')
rfm.head()

In [ ]:
# Compute the maximum date to know the last transaction date
max_date = max(df['InvoiceDate'])

# Compute the difference between max date and transaction date
df['Diff'] = max_date - df['InvoiceDate']

# Compute last transaction date to get the recency of customers
rfm_p = df.groupby('CustomerID')['Diff'].min().reset_index()

## Extract number of days only
rfm_p['Diff'] = rfm_p['Diff'].dt.days 

rfm = pd.merge(rfm, rfm_p, on='CustomerID', how='inner')

In [ ]:
rfm = rfm.rename(columns={'Diff':'Recency'})
rfm.head()

In [ ]:
rfm.describe()

In [ ]:
sns.heatmap(rfm.corr(), annot=True)

- There is a negative correlation between Recency-Frequency and Recency-Monetary, but there is a positive correlation between Frequency-Monetary.Low value of recency means less recent transactions while high frequency means more frequent transactions and high Monetary value means high spending on transactions.

In [ ]:
# Outlier Analysis of Amount, Frequency and Recency

features = ['Amount','Frequency','Recency']
plt.figure(figsize=(5,4))
sns.boxplot(data = rfm[features])
plt.title("Outliers  Distribution")
plt.show()

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,3,1)
sns.distplot( rfm['Amount'], kde=True)

plt.subplot(1,3,2)
sns.distplot( rfm['Frequency'], kde=True)

plt.subplot(1,3,3)
sns.distplot( rfm['Recency'], kde=True)
plt.show()

In [ ]:
print(rfm['Amount'].skew() )
print(rfm['Frequency'].skew())
print(rfm['Recency'].skew())

In [ ]:
rfm['Amount'] = np.log(rfm['Amount'] + 0.01)
rfm['Frequency'] = np.log(rfm['Frequency'] + 0.01)
rfm['Recency'] = np.log(rfm['Recency'] + 0.01)

In [ ]:
print(a.skew())
print(b.skew())
print(c.skew())

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,3,1)
sns.distplot( rfm['Amount'], kde=True)

plt.subplot(1,3,2)
sns.distplot( rfm['Frequency'], kde=True)

plt.subplot(1,3,3)
sns.distplot( rfm['Recency'], kde=True)
plt.show()

In [ ]:
print(rfm['Amount'].skew() )
print(rfm['Frequency'].skew())
print(rfm['Recency'].skew())

In [ ]:
rfm.head()

In [ ]:
X = rfm.iloc[:, 1:]

In [ ]:
X.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

### Modelling(K-Means Clutering)

**Elbow Curve**

- **Find the optimal number of cluster.**

In [ ]:
from sklearn.cluster import KMeans

wcss = []
for k in range(1,7):
    k_model = KMeans(n_clusters = k, init = 'k-means++',random_state=0)
    k_model.fit(X)
    wcss.append(k_model.inertia_)
print(wcss)

In [ ]:
plt.plot(range(1,7),wcss, '-o')
plt.xticks(range(1,7))
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

- Optimal number of clusters is 3.

### K-Means Model

In [ ]:
#Modelling
kmeans = KMeans(n_clusters = 3, init = 'k-means++', random_state=0)

#Predict
y_kmeans = kmeans.fit_predict(X)
print(y_kmeans)

In [ ]:
rfm['cluster'] = pd.DataFrame(y_kmeans)
rfm.head()

In [ ]:
rfm[rfm['cluster']==1]

In [ ]:
rfm['cluster'].value_counts(normalize=True)

In [ ]:
# get centroids
centroids = kmeans.cluster_centers_
cen_x = [i[0] for i in centroids] 
cen_y = [i[1] for i in centroids]

## add to dataframe
rfm['cen_x'] = rfm.cluster.map({0:cen_x[0], 1:cen_x[1], 2:cen_x[2]})
rfm['cen_y'] = rfm.cluster.map({0:cen_y[0], 1:cen_y[1], 2:cen_y[2]})

# define and map colors
colors = ['#DF2020', '#81DF20', '#2095DF']
rfm['c'] = rfm.cluster.map({0:colors[0], 1:colors[1], 2:colors[2]})

# Plot clusters
plt.figure(figsize=(12,5))
plt.scatter(rfm.Frequency, rfm.Amount, c=rfm.cluster, alpha = 0.6, s=25)
plt.xlabel('Frequency', fontsize=15)
plt.ylabel('Monetary', fontsize=15)

In [ ]:
customer_seg = rfm['cluster'].value_counts().index
customer_val = rfm['cluster'].value_counts().values

plt.pie(customer_val, labels = ['High valued customer','Average valued customer','Low valued customer'], autopct = '%1.2f%%')
plt.show()

This plot clearly indicates that green cluster is high value customer cohort (Cluster 0) and blue cluster is Lost/low value customer cohort (Cluster 1) while yellow cluster is average value customer cohort (Cluster 2)
High Value :- Frequent spending customers with high monetary transactions and had transacted recently

Low Value/ Lost :- Customers with least frequency and monetary spending and made purchase long time ago.

Average Value :- Customers who made their transactions some time ago with less frequency and monetary value. 